In [1]:
from tradingview_ta import TA_Handler, Interval, Exchange , TradingView
import Coinex_API_Class as CoinexAPI
import pandas as pd
from tqdm import tqdm
%reload_ext autoreload
%autoreload 2
    
api_key = "47D7C4B286224298BB3D88A9D7161A45"
api_secret = "F0F2D4156F6892ED2F03576FEE74CFA658C5F8271DCE102D"
coinex = CoinexAPI.Coinex_API(api_key,api_secret)

def prepare_data (period,limit) :
    data_spot = coinex.get_spot_cum_ret(period,limit)
    data_spot['Recomandation'] = None
    data_spot['Buy'] = None
    data_spot['Sell'] = None
    data_spot['Neutral'] = None
    if period== "1min" : preiod_ta = "INTERVAL_1_MINUTE"
    elif period == "5min" : preiod_ta = "INTERVAL_5_MINUTES"
    elif period == "15min" : preiod_ta = "INTERVAL_15_MINUTES"
    elif period == "30min" : preiod_ta = "INTERVAL_30_MINUTES"
    elif period == "1hour" : preiod_ta = "INTERVAL_1_HOUR"
    elif period == "2hour" : preiod_ta = "INTERVAL_2_HOUR"
    elif period == "4hour" : preiod_ta = "INTERVAL_4_HOUR"
    elif period == "1day" : preiod_ta = "INTERVAL_1_DAY"
    elif period == "1week" : preiod_ta = "INTERVAL_1_WEEK"
    
    for index, row in tqdm(data_spot.iterrows(), total=len(data_spot), desc="Processing symbol" , position=0):
        try : 
            if period== "1min" : 
                data= (TA_Handler(symbol=row['symbol'], screener="crypto", exchange="COINEX", interval=Interval.INTERVAL_1_MINUTE)).get_analysis().summary
            elif period == "5min" : 
                data= (TA_Handler(symbol=row['symbol'], screener="crypto", exchange="COINEX", interval=Interval.INTERVAL_5_MINUTES)).get_analysis().summary
            elif period == "15min" : 
                data= (TA_Handler(symbol=row['symbol'], screener="crypto", exchange="COINEX", interval=Interval.INTERVAL_15_MINUTES)).get_analysis().summary
            elif period == "30min" : 
                data= (TA_Handler(symbol=row['symbol'], screener="crypto", exchange="COINEX", interval=Interval.INTERVAL_30_MINUTES)).get_analysis().summary
            elif period == "1hour" : 
                data= (TA_Handler(symbol=row['symbol'], screener="crypto", exchange="COINEX", interval=Interval.INTERVAL_1_HOUR)).get_analysis().summary
            elif period == "2hour" : 
                data= (TA_Handler(symbol=row['symbol'], screener="crypto", exchange="COINEX", interval=Interval.INTERVAL_2_HOURS)).get_analysis().summary
            elif period == "4hour" : 
                data= (TA_Handler(symbol=row['symbol'], screener="crypto", exchange="COINEX", interval=Interval.INTERVAL_4_HOURS)).get_analysis().summary
            elif period == "1day" : 
                data= (TA_Handler(symbol=row['symbol'], screener="crypto", exchange="COINEX", interval=Interval.INTERVAL_1_DAY)).get_analysis().summary
            elif period == "1week" : 
                data= (TA_Handler(symbol=row['symbol'], screener="crypto", exchange="COINEX", interval=Interval.INTERVAL_1_WEEK)).get_analysis().summary
            data_spot.loc[index, 'Recomandation'] =data['RECOMMENDATION']
            data_spot.loc[index, 'Buy'] = data['BUY']
            data_spot.loc[index, 'Sell'] = data['SELL']
            data_spot.loc[index, 'Neutral'] = data['NEUTRAL']
        except Exception as e:
            continue
        df=data_spot.drop(columns=["Open","High","Low","Return"] , axis=1)
    return df

In [2]:
df= prepare_data("15min",48)  # For 12 Hours ago
df.to_csv('data.csv', index=False)

Processing symbol: 100%|███████████████████████████████████████████████████████████| 1173/1173 [10:17<00:00,  1.90it/s]


In [3]:
df

,Time,Close,Cum_Return,Volume,Value,symbol,period,limit,maker_fr,taker_fr,Recomandation,Buy,Sell,Neutral
0,2024-09-29 22:15:00,0.000879,1.023446,40650.000000,35.734945,NEOXUSDT,15min,48,0.003,0.003,BUY,15,1,10
1,2024-09-29 22:15:00,0.032635,1.073696,395.500000,12.902990,HGPTUSDT,15min,48,0.003,0.003,BUY,14,2,10
2,2024-09-29 22:15:00,0.003958,1.031252,15889.903199,63.180803,ATLASUSDT,15min,48,0.003,0.003,BUY,15,1,10
3,2024-09-29 22:15:00,0.289300,0.993817,40.050000,11.599035,WILDUSDT,15min,48,0.003,0.003,BUY,12,5,9
4,2024-09-29 22:15:00,0.037243,1.014326,406.500000,15.175007,WAXPUSDT,15min,48,0.002,0.002,BUY,10,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,2024-09-29 22:30:00,0.072042,1.010860,173.809045,12.522647,BSWUSDT,15min,48,0.003,0.003,NEUTRAL,8,11,7
1169,2024-09-29 22:30:00,0.016011,1.037721,457.663201,7.328343,IRISUSDT,15min,48,0.003,0.003,BUY,12,5,9
1170,2024-09-29 22:30:00,4.786000,1.027325,1.690055,8.092703,FLXUSDT,15min,48,0.003,0.003,BUY,12,6,8
1171,2024-09-29 22:30:00,0.408900,1.093900,16.759845,6.854221,DNXUSDT,15min,48,0.003,0.003,BUY,14,3,9


In [8]:
df = pd.read_csv("data.csv")
df2 = df[(df['Recomandation'] == "STRONG_BUY") | (df['Recomandation'] == "STRONG_SELL")]
df2 = df2.sort_values(['Cum_Return'],ascending=False)
df2 = df2.reindex(['Time', 'period', 'limit','Recomandation','symbol','Cum_Return','Close','Volume','Value','maker_fr','taker_fr','Buy','Sell','Neutral',], axis=1)

In [9]:
df2.head(10)

,Time,period,limit,Recomandation,symbol,Cum_Return,Close,Volume,Value,maker_fr,taker_fr,Buy,Sell,Neutral
893,2024-09-29 22:30:00,15min,48,STRONG_BUY,USEDCARUSDT,1.291667,0.006758,0.000000e+00,0.000000,0.003,0.003,17.0,0.0,9.0
1094,2024-09-29 22:30:00,15min,48,STRONG_BUY,ANALOSUSDT,1.169978,0.000049,2.702317e+06,133.091548,0.003,0.003,17.0,1.0,8.0
879,2024-09-29 22:30:00,15min,48,STRONG_BUY,USTCUSDT,1.166956,0.024198,0.000000e+00,0.000000,0.003,0.003,17.0,1.0,8.0
731,2024-09-29 22:15:00,15min,48,STRONG_BUY,PONKEUSDT,1.143470,0.373000,4.343387e+02,161.222097,0.003,0.003,17.0,1.0,8.0
847,2024-09-29 22:15:00,15min,48,STRONG_BUY,HMNDUSDT,1.137452,0.120091,1.336130e+03,162.646493,0.003,0.003,16.0,0.0,10.0
42,2024-09-29 22:15:00,15min,48,STRONG_BUY,TOOKERUSDT,1.129446,0.025975,9.250000e+02,24.036190,0.003,0.003,16.0,1.0,9.0
589,2024-09-29 22:15:00,15min,48,STRONG_SELL,MIGGLESUSDT,1.120442,0.026578,1.405144e+03,37.439366,0.003,0.003,1.0,16.0,9.0
147,2024-09-29 22:15:00,15min,48,STRONG_BUY,SNEKUSDT,1.114588,0.001061,1.219780e+05,129.094818,0.003,0.003,18.0,3.0,5.0
1060,2024-09-29 22:30:00,15min,48,STRONG_BUY,DJTUSDT,1.097959,0.000269,1.144763e+04,3.079413,0.003,0.003,16.0,0.0,10.0
701,2024-09-29 22:15:00,15min,48,STRONG_BUY,RAILUSDT,1.096358,0.993300,3.137167e+01,31.104833,0.003,0.003,17.0,1.0,8.0
